<a href="https://colab.research.google.com/github/RhysWangJunfei/Optimization-Leaks-Inferring-Private-Parameters-in-Optimization-Based-Decision-Systems-/blob/main/usenix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
from scipy.io import loadmat
import pdb;
import math

In [4]:
conditions = loadmat('/content/drive/MyDrive/Postdoc/OPF/LMP_Attack_Case118/conditions30.mat')['conditions_list']
solutions = loadmat('unbinding_solutions30_10percent.mat')['datapoints_list']
mat_gen = loadmat('/content/drive/MyDrive/Postdoc/OPF/LMP_Attack_Case118/gen30.mat')['gen30']
gen_ids = mat_gen[:,0]-1
gencost = loadmat('/content/drive/MyDrive/Postdoc/OPF/LMP_Attack_Case118/gencost30.mat')['gencost30']

In [7]:
grid_size = 30
gen_size = 6

In [15]:
true_a = gencost[:,4]
true_b = gencost[:,5]

In [17]:
true_b

array([2.  , 1.75, 1.  , 3.25, 3.  , 3.  ])

In [18]:
indices = np.array([1,2,3,4,5])
aaa=true_a[indices]
bbb=true_b[indices]
g_index_list =indices
Pg_list = []
LMP_list = []
PG_aggregate = np.zeros([solutions.shape[0]])
naive_index = 0
for g_index in g_index_list:
  a_i = aaa[naive_index]
  b_i = bbb[naive_index]
  gg = int(gen_ids[g_index])
  Pg = solutions[:,grid_size*2+g_index]
  Pg_list.append(Pg)
  PG_aggregate = PG_aggregate+Pg
  LMP = 2*a_i*Pg+b_i
  LMP_list.append(LMP)
  naive_index = naive_index+1
LMP_arr = np.array(LMP_list).T
Pg_arr = np.array(Pg_list).T

In [19]:
LMP=np.asarray(LMP_list).T
PG=PG_aggregate

In [20]:
LMPT = LMP_arr.T

In [25]:
import numpy as np
#adv_points = [None] * N_b
def check_weighted_conditions_vec(LMP, PG, a_max=0.5, a_min=0.008, chunk_size=500):
    K, N_b = LMP.shape
    best_cond = np.ones(N_b) * np.inf
    for start in range(0, K, chunk_size):
        print(start)
        #pdb.set_trace()
        end = min(K, start + chunk_size)
        ΔLMP_chunk = LMP[start:end, None, :] - LMP[None, :, :]   # (chunk, K, N_b)
        ΔPG_chunk  = PG[start:end, None] - PG[None, :]           # (chunk, K)

        abs_LMP = np.abs(ΔLMP_chunk)
        mask_pos = ΔLMP_chunk > 0
        mask_neg = ΔLMP_chunk < 0

        C_exe_pos = np.sum(ΔLMP_chunk * mask_pos, axis=2)
        C_exe_neg = np.sum(ΔLMP_chunk * mask_neg, axis=2)
        conservative_factor=1
        # Loop over buses i
        for i in range(N_b):
            if best_cond[i]<aaa[i]:
              continue
            A = ΔLMP_chunk[:, :, i]
            B = ΔPG_chunk
            weight = np.ones(N_b)
            weight[i]=0
            C_excl_i = np.sum(abs_LMP*weight.reshape([1,1,-1]), axis=2)
            # Mask valid regions (A>0,B>0)
            #mask_valid = (A > 0) & (B > 0)
            #if not np.any(mask_valid):
            #    continue

            # Exclude current bus i
            #C_excl_i = C_sum - abs_LMP[:, :, i]
            C_exe_pos_i = C_exe_pos - np.where(mask_pos[:, :, i], ΔLMP_chunk[:, :, i], 0)
            C_exe_neg_i = C_exe_neg - np.where(mask_neg[:, :, i], ΔLMP_chunk[:, :, i], 0)

            #Define D_i
            D_i1 = B+0.5*C_exe_pos_i/a_max+0.5*C_exe_neg_i/a_min
            D_i2 = -1*(B+0.5*C_exe_pos_i/a_min+0.5*C_exe_neg_i/a_max)
            case_i1 = np.where((D_i1>0))
            case_i2 = np.where((D_i2>0))

            #in case 2:
            num_cases = case_i2[0].shape
            D_i2_min = D_i2[case_i2]
            D_i2_max = -1*(B[case_i2]+0.5*C_exe_pos_i[case_i2]/a_max+0.5*C_exe_neg_i[case_i2]/a_min)
            cond2 = (np.sqrt(abs(A[case_i2]) * C_excl_i[case_i2]) + C_exe_pos_i[case_i2]) / (-2 * B[case_i2] - C_exe_neg_i[case_i2] / a_max)
            valid_self = (-(A[case_i2]/ D_i2_max) >= a_min/conservative_factor)&(-(A[case_i2]/ D_i2_min)<=a_max*conservative_factor)&(cond2>a_min)
            #pdb.set_trace()
                        # Update best
            cond_valid = np.where(valid_self, cond2, np.inf)
            min_val = np.min(cond_valid)
            if np.isfinite(min_val) and min_val < best_cond[i]:
                best_cond[i] = min_val
                #pdb.set_trace()
                agm = np.argmin(cond_valid)
                α_rel, β_idx = case_i2[0][agm], case_i2[1][agm]
                adv_points[i] = (start + α_rel, β_idx)
                print(best_cond)

            #in case 1:
            num_cases = case_i1[0].shape
            D_i1_min = D_i1[case_i1]
            D_i1_max = B[case_i1]+0.5*C_exe_pos_i[case_i1]/a_min+0.5*C_exe_neg_i[case_i1]/a_max
            cond1 = ((np.sqrt(abs(A[case_i1]) * C_excl_i[case_i1]) - C_exe_neg_i[case_i1]) / (2 * B[case_i1] + C_exe_pos_i[case_i1] / a_max))
            valid_self = ((A[case_i1]/ D_i1_max) >= a_min/conservative_factor)&((A[case_i1]/ D_i1_min)<=a_max*conservative_factor)&(cond1>a_min)
            cond_valid = np.where(valid_self, cond1, np.inf)
            if cond_valid.shape[0]>0:
              min_val = np.min(cond_valid)
              if np.isfinite(min_val) and min_val < best_cond[i]:
                  best_cond[i] = min_val
                  #pdb.set_trace()
                  agm = np.argmin(cond_valid)
                  α_rel, β_idx = case_i1[0][agm], case_i1[1][agm]
                  adv_points[i] = (start + α_rel, β_idx)
                  print(best_cond)

            del valid_self,cond_valid,cond2, D_i1, D_i2,case_i1,case_i2,C_excl_i,D_i2_min,D_i2_max
        del ΔLMP_chunk, ΔPG_chunk, abs_LMP, C_exe_pos, C_exe_neg
    return best_cond, adv_points


In [ ]:
aaa

array([0.01      , 0.11764706, 0.01      , 0.01      , 0.01      ,
       0.01      , 0.01      , 0.01      , 0.04901961, 0.20833333,
       0.01      , 0.06451613, 0.0625    , 0.02557545])

In [26]:
ttt = np.asarray(LMP_list).T
K, N_b = ttt.shape
adv_points = [None] * N_b
best_cond, adv_points = check_weighted_conditions_vec(np.asarray(LMP_list).T, PG_aggregate, a_max=0.21,a_min=0.008,chunk_size=100)

0
[0.00857074        inf        inf        inf        inf]
[0.00857074 0.01001476        inf        inf        inf]
[0.00857074 0.00801879        inf        inf        inf]
[0.00857074 0.00801879 0.0080003         inf        inf]
[0.00857074 0.00801879 0.0080003  0.00911971        inf]
[0.00857074 0.00801879 0.0080003  0.00911411        inf]
[0.00857074 0.00801879 0.0080003  0.00911411 0.00838433]
100
200
300
400
500
600
700
800
900


In [27]:
def fa(adv_points,a_list):
  new_a_list = np.zeros_like(a_list)
  for i in range(0,len(adv_points)):
    #pdb.set_trace()
    beta_i = adv_points[i][0]
    alpha_i = adv_points[i][1]
    A = LMP_arr[beta_i,i]-LMP_arr[alpha_i,i]
    B = PG_aggregate[beta_i]-PG_aggregate[alpha_i]
    C=0
    for g in range(0,LMP_arr.shape[1]):
      if g!=i:
        c = (LMP_arr[alpha_i,g]-LMP_arr[beta_i,g])/(2*a_list[g])
        C = C+c
    #pdb.set_trace()
    new_a_list[i] = A/(2*(B+C))
    #print(B+C)
  return new_a_list

In [28]:
a_list = [0.01]*g_index_list.shape[0]
for it in range(1,100):
  a_list = fa(adv_points,a_list)
  print(a_list)

[0.04109694 0.04609128 0.0093342  0.05246834 0.04281446]
[0.0152344  0.03211845 0.00865868 0.01806521 0.01877414]
[0.01991755 0.05752724 0.00853997 0.02840802 0.02747817]
[0.01695315 0.05184212 0.00840339 0.02306451 0.02331291]
[0.01797857 0.06150593 0.00838291 0.02569074 0.02551484]
[0.01737333 0.05980532 0.00835311 0.02454444 0.02460531]
[0.01760411 0.06231752 0.00834926 0.0251537  0.02511546]
[0.01747085 0.06188939 0.00834274 0.02489698 0.0249109 ]
[0.01752312 0.0624681  0.008342   0.02503492 0.02502635]
[0.0174933  0.06236529 0.00834057 0.02497689 0.02498004]
[0.01750516 0.06249479 0.00834043 0.02500797 0.02500604]
[0.01749846 0.06247044 0.00834012 0.02499482 0.02499553]
[0.01750115 0.06249925 0.00834009 0.02500182 0.02500138]
[0.01749965 0.06249352 0.00834003 0.02499884 0.024999  ]
[0.01750026 0.06249992 0.00834002 0.02500041 0.02500032]
[0.01749992 0.06249858 0.00834001 0.02499974 0.02499978]
[0.01750006 0.0625     0.00834    0.02500009 0.02500007]
[0.01749998 0.06249969 0.00834 

In [30]:
true_a[indices]

array([0.0175 , 0.0625 , 0.00834, 0.025  , 0.025  ])